# IBM Applied Data Science Capstone

## Segmenting & Clustering Toronto Neighborhoods: Assignment Part III
Explore and cluster the neighborhoods in Toronto; generate maps and report observations.

### Install/import Python packages/libraries

In [1]:
#!pip install geopy
#!pip install folium
import folium
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import json, requests
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

### Load the combined dataframe (from csv file)

In [2]:
dfTM=pd.read_csv('Toronto_geodataframe.csv')
dfTM.drop(labels='Unnamed: 0',axis=1,inplace=True)
dfTM.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Get the latitude and longitude of Toronto

In [3]:
geolocator=Nominatim(user_agent='my_app')
location=geolocator.geocode('Toronto')
latitude=location.latitude
longitude=location.longitude
print('The geograpical coordinates of Toronto are: {} latitude, {} longitude'.format(latitude,longitude))

The geograpical coordinates of Toronto are: 43.653963 latitude, -79.387207 longitude


### Create a map of Toronto with neighborhoods superimposed

In [4]:
mapT=folium.Map(location=[latitude,longitude],zoom_start=10)
for lat, lng, borough, neighborhood in zip(dfTM['Latitude'],dfTM['Longitude'],dfTM['Borough'],dfTM['Neighborhood']):
    label='{}, {}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(mapT)   
mapT

### Focus on boroughs containing "Toronto"

In [5]:
borough_names=list(dfTM.Borough.unique())
borough_Toronto=[]
for x in borough_names:
    if "toronto" in x.lower():
        borough_Toronto.append(x)
dfTT=dfTM[dfTM['Borough'].isin(borough_Toronto)].reset_index(drop=True)
print('There are {} neighborhoods in {} boroughs:'.format(dfTT.shape[0],len(borough_Toronto)),borough_Toronto)
dfTT.head()

There are 38 neighborhoods in 4 boroughs: ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Create a map of Toronto with the selected neighborhoods superimposed

In [6]:
mapTT=folium.Map(location=[latitude,longitude],zoom_start=10)
for lat, lng, borough, neighborhood in zip(dfTT['Latitude'],dfTT['Longitude'],dfTT['Borough'],dfTT['Neighborhood']):
    label='{}, {}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(mapTT)   
mapTT

### Use Foursquare API to explore the neighborhoods

In [7]:
CLIENT_ID='your Foursquare ID'
CLIENT_SECRET='your Foursquare Secret'
VERSION='20180605'
print('Client ID:',CLIENT_ID)
print('Client PW:',CLIENT_SECRET)
print('Version:',VERSION)

Client ID: your Foursquare ID
Client PW: your Foursquare Secret
Version: 20180605


### Get a dataframe of the top 100 venues per postal code that are within a radius of 500m

In [8]:
radius=500; LIMIT=100; venues=[]
for lat, long, post, borough, neighborhood in zip(dfTT['Latitude'],dfTT['Longitude'],dfTT['PostalCode'],dfTT['Borough'],dfTT['Neighborhood']):
    url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}' \
        .format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,radius,LIMIT)   
    results=requests.get(url).json()["response"]['groups'][0]['items']    
    for venue in results:
        venues.append((post,borough,neighborhood,lat,long,venue['venue']['name'],
            venue['venue']['location']['lat'],venue['venue']['location']['lng'],venue['venue']['categories'][0]['name']))
dfTV=pd.DataFrame(venues)
dfTV.columns=['PostalCode','Borough','Neighborhood','BoroughLatitude','BoroughLongitude','VenueName','VenueLatitude','VenueLongitude','VenueCategory']
print('There are {} venues of {} categories.'.format(dfTV.shape[0],len(dfTV['VenueCategory'].unique())))
dfTV.head()

There are 1700 venues of 236 categories.


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,St-Denis Studios Inc.,43.675031,-79.288022,Music Venue
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


### Check how many venues were returned per postal code

In [9]:
dfTV.groupby(['PostalCode','Borough','Neighborhood']).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale",44,44,44,44,44,44
M4L,East Toronto,"The Beaches West, India Bazaar",22,22,22,22,22,22
M4M,East Toronto,Studio District,37,37,37,37,37,37
M4N,Central Toronto,Lawrence Park,4,4,4,4,4,4
M4P,Central Toronto,Davisville North,7,7,7,7,7,7
M4R,Central Toronto,North Toronto West,20,20,20,20,20,20
M4S,Central Toronto,Davisville,32,32,32,32,32,32
M4T,Central Toronto,"Moore Park, Summerhill East",3,3,3,3,3,3


### Analyze the venues per postal code using one hot encoding 

In [10]:
pd.set_option('display.max_columns',None); pd.set_option('display.max_rows',None)
toronto_ohe=pd.get_dummies(dfTV[['VenueCategory']],prefix='',prefix_sep='')
toronto_ohe['PostalCode']=dfTV['PostalCode']
toronto_ohe['Borough']=dfTV['Borough']
toronto_ohe['Neighborhoods']=dfTV['Neighborhood']
toronto_ohe=toronto_ohe[list(toronto_ohe.columns[-3:]) + list(toronto_ohe.columns[:-3])]
toronto_ohe.head()

,PostalCode,Borough,Neighborhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Get the mean frequency of occurrence of each venue category per postal code

In [11]:
toronto_area=toronto_ohe.groupby(['PostalCode','Borough','Neighborhoods']).mean().reset_index()
toronto_area.head()

,PostalCode,Borough,Neighborhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000

### Get the 3 most common venues per postal code

In [12]:
num_top_venues=3; indicators=['st','nd','rd']
areaColumns=['PostalCode','Borough','Neighborhoods']; freqColumns=[]
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Venue'.format(ind+1,indicators[ind]))
    except:
        freqColumns.append('{}th Venue'.format(ind+1))
columns=areaColumns+freqColumns
area_venues_sorted=pd.DataFrame(columns=columns)
area_venues_sorted['PostalCode']=toronto_area['PostalCode']
area_venues_sorted['Borough']=toronto_area['Borough']
area_venues_sorted['Neighborhoods']=toronto_area['Neighborhoods']
for ind in np.arange(toronto_area.shape[0]):
    row_categories=toronto_area.iloc[ind, 1:].iloc[3:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    area_venues_sorted.iloc[ind, 3:]=row_categories_sorted.index.values[0:num_top_venues]
area_venues_sorted

,PostalCode,Borough,Neighborhoods,1st Venue,2nd Venue,3rd Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Pub,Neighborhood
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",Park,Sandwich Place,Pet Store
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Bakery
4,M4N,Central Toronto,Lawrence Park,Construction & Landscaping,Park,Swim School
5,M4P,Central Toronto,Davisville North,Hotel,Gym,Sandwich Place
6,M4R,Central Toronto,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio
7,M4S,Central Toronto,Davisville,Sandwich Place,Dessert Shop,Pizza Place
8,M4T,Central Toronto,"Moore Park, Summerhill East",Playground,Tennis Court,Trail
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Coffee Shop,Pub,Pizza Place


### Group areas into 3 clusters (using k-means algorithm) per venue category

In [13]:
k=3; area_clusters=toronto_area.drop(['PostalCode','Borough','Neighborhoods'],1)
kmeans=KMeans(n_clusters=k,random_state=0).fit(area_clusters)
area_clustered=dfTT.copy()
area_clustered['Cluster']=kmeans.labels_+1
area_clustered=area_clustered.join(area_venues_sorted.drop(['Borough','Neighborhoods'],1).set_index('PostalCode'),on='PostalCode')
area_clustered.sort_values(['Cluster'],inplace=True)
area_clustered.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Venue,2nd Venue,3rd Venue
10,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,1,Playground,Tennis Court,Trail
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Pub,Neighborhood
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,2,Coffee Shop,Café,Hotel
23,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,2,Trail,Sushi Restaurant,Jewelry Store


### Create a map of Toronto with the clusters superimposed

In [14]:
map_clusters=folium.Map(location=[latitude,longitude],zoom_start=11)
x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]
markers_colors=[]
for lat, lon, post, bor, poi, cluster in zip(area_clustered['Latitude'],area_clustered['Longitude'],area_clustered['PostalCode'],area_clustered['Borough'],area_clustered['Neighborhood'],area_clustered['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor,post,poi,cluster),parse_html=True)
    folium.CircleMarker([lat, lon],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
map_clusters

### Cluster 1

In [15]:
area_clustered.loc[area_clustered['Cluster']==1,area_clustered.columns[[1]+list(range(5,area_clustered.shape[1]))]]

,Borough,Cluster,1st Venue,2nd Venue,3rd Venue
10,Downtown Toronto,1,Park,Playground,Trail
8,Central Toronto,1,Playground,Tennis Court,Trail


### Cluster 2

In [16]:
area_clustered.loc[area_clustered['Cluster']==2,area_clustered.columns[[1]+list(range(5,area_clustered.shape[1]))]]

,Borough,Cluster,1st Venue,2nd Venue,3rd Venue
0,East Toronto,2,Health Food Store,Pub,Neighborhood
21,Downtown Toronto,2,Coffee Shop,Café,Hotel
23,Central Toronto,2,Trail,Sushi Restaurant,Jewelry Store
24,Central Toronto,2,Coffee Shop,Café,Sandwich Place
25,Downtown Toronto,2,Café,Restaurant,Bakery
26,Downtown Toronto,2,Café,Vegetarian / Vegan Restaurant,Bar
27,Downtown Toronto,2,Airport Service,Airport Terminal,Airport Lounge
28,Downtown Toronto,2,Coffee Shop,Restaurant,Café
29,Downtown Toronto,2,Coffee Shop,Café,Hotel
30,Downtown Toronto,2,Grocery Store,Café,Park


### Cluster 3

In [17]:
area_clustered.loc[area_clustered['Cluster']==3,area_clustered.columns[[1]+list(range(5,area_clustered.shape[1]))]]

,Borough,Cluster,1st Venue,2nd Venue,3rd Venue
22,Central Toronto,3,Garden,Yoga Studio,Farmers Market


### Observations

Most of the neighborhoods fall into Cluster 2 and are business areas with shops, cafè and standard restaurants in East, West, Central and Downtown Toronto; Cluster 3 is just a garden close to a yoga studio and farmers market in Central Toronto; Cluster 1 is mostly parks, playgrounds and trails in Central and Downtown Toronto. As these cluters are rather homogeneous and their type is well defined, no additional clusters are hereby sought (unless other criteria are introduced).